# Intruduction to NLP fundamentals in TensorFlow
NLP has the goal of deriving information out of natural language (could be sequences of text or speech).

Another common term for NLP problems is sequence to sequence problems (seq2seq).

## Check for the GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-d37683d2-68cc-714a-abd0-1cffcfd60b7c)


## Get helper functions

In [2]:
# Download the helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-03-18 21:56:22--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-03-18 21:56:22 (67.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset
The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labeled as disaster or not disaster).

See the orignal source here: https://www.kaggle.com/competitions/nlp-getting-started

In [3]:
# Download the data
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data('nlp_getting_started.zip')

--2023-03-18 21:56:28--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.196.128, 173.194.197.128, 173.194.192.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.196.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2023-03-18 21:56:28 (116 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Become one with the data - visualize a text dataset

To visualize our text sample, we first need to read them in. One way to do so would be to use Python.

But we can get visual straight away.

So another way to do this is to use Pandas.

TF help: https://www.tensorflow.org/tutorials/load_data/text

In [4]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [6]:
train_df['text'][1]

'Forest fire near La Ronge Sask. Canada'

In [7]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, 
                                    random_state=42)

train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
# What does the test dataset look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# How many examples are there?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

If dataset is imbalanced: https://www.tensorflow.org/tutorials/structured_data/imbalanced_data

In [10]:
# How many total samples
len(train_df), len(test_df)

(7613, 3263)

In [11]:
# Let's visualize some random training examples
import random 
random_index = random.randint(0, len(train_df)-5) #create random indexes not higher then random number of samples

for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text: \n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text: 
One Year on from the Sinjar Massacre #Yazidis Blast Lack of Action Over Hostages
http://t.co/BAqOcMcJqc

---

Target: 1 (real disaster)
Text: 
@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao

---

Target: 0 (not real disaster)
Text: 
Whirlwind weekend #1 starts in 12 hours #cantwaittoplayinminneapolis

---

Target: 0 (not real disaster)
Text: 
@SunderCR two hours of Sandstorm remixes. All merged together. No between-song silence.

---

Target: 1 (real disaster)
Text: 
Governor Allows Parole for School Bus Hijacker http://t.co/DzlPNP399x

---



### Split data into training and validation sets

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
# Use train_test_split to split the training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'],
                                                                            train_df_shuffled['target'],
                                                                            test_size=0.1,
                                                                            random_state=42)

In [14]:
# Check the lenghts
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [15]:
# Check some top samples
train_sentences[:10], train_labels[:10]

(5921    @mogacola @zamtriossu i screamed after hitting...
 3904              Imagine getting flattened by Kurt Zouma
 2804    @Gurmeetramrahim #MSGDoing111WelfareWorks Gree...
 3718    @shakjn @C7 @Magnums im shaking in fear he's g...
 1667    Somehow find you and I collide http://t.co/Ee8...
 4435    @EvaHanderek @MarleyKnysh great times until th...
 2544                     destroy the free fandom honestly
 7223    Weapons stolen from National Guard Armory in N...
 4265    @wfaaweather Pete when will the heat wave pass...
 6568    Patient-reported outcomes in long-term survivo...
 Name: text, dtype: object, 5921    0
 3904    0
 2804    1
 3718    0
 1667    0
 4435    1
 2544    1
 7223    0
 4265    1
 6568    1
 Name: target, dtype: int64)

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text into numbers.

There are a few ways to do this, namely:
* Tokenization - direct mapping of token (a token could be a word or a character) to number
* Embeddings - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embeddings can be learned)

### Text vectorizaton - tokenization

In [16]:
train_sentences[:5]

5921    @mogacola @zamtriossu i screamed after hitting...
3904              Imagine getting flattened by Kurt Zouma
2804    @Gurmeetramrahim #MSGDoing111WelfareWorks Gree...
3718    @shakjn @C7 @Magnums im shaking in fear he's g...
1667    Somehow find you and I collide http://t.co/Ee8...
Name: text, dtype: object

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [18]:
# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (automaticall add <OOV>)
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None, # create groups of n-words?
                                    output_mode='int', # how to map tokens to numbers
                                    output_sequence_length=None, # how long you want your sequences to be
                                    #pad_to_max_tokens=True
                                    )

In [19]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [20]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words in vocabulary
max_length = 15 # max length our sequences will be (e.g. for many words will a model see)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [21]:
# Fit the text vextorizer to the training set
text_vectorizer.adapt(train_sentences)

In [22]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [23]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
      \n\nVectorized version: ")
text_vectorizer([random_sentence])

Original text:
 CommoditiesåÊAre Crashing Like It's 2008 All Over Again http://t.co/nJD1N5TxXe via @business      

Vectorized version: 


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1, 3965,   25,   37, 6345,   44,   60,  282,    1,   49,  691,
           0,    0,    0,    0]])>

In [24]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in training data
top_5_words = words_in_vocab[:5] # most 5 common words
bottom_5_words = words_in_vocab[-5:]

print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words are: {top_5_words}")
print(f"Top 5 least common words are: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words are: ['', '[UNK]', 'the', 'a', 'in']
Top 5 least common words are: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Create and Embedding using and Embedding Layer
To make our embedding we're going to use TensorFlow's Embedding Layer.

Parameters we care the most are:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example: a value of 100 would mean each token gets represented by a vector 100 long
* `input_length` = length of sequences being passes to the embedding layer

In [25]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # output shape
                             input_length=max_length #how long in each input
                             )

embedding

In [26]:
# Get another random sentence
random_sentence = random.choice(train_sentences)
print(f"Original text: {random_sentence}")
print(f"Embedded version: ")

# Embed the random sentence (turn it into the dense vector of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text: Pic of 16yr old PKK suicide bomber who detonated bomb in Turkey Army trench released http://t.co/uJNRGFAnGj http://t.co/TmIpEgQyeV
Embedded version: 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02668208, -0.04364238, -0.0132379 , ...,  0.00757761,
          0.03587512, -0.01495224],
        [ 0.01938936, -0.00547592, -0.04855162, ...,  0.03776148,
          0.01300493,  0.00653641],
        [-0.02029852, -0.03666019,  0.00510357, ...,  0.04987532,
          0.0456816 ,  0.04807875],
        ...,
        [ 0.01333654, -0.02785575,  0.04120138, ...,  0.02981361,
          0.04038855, -0.02822437],
        [ 0.03151568,  0.03347132, -0.03018901, ..., -0.02103362,
          0.03999421, -0.02940708],
        [ 0.0094077 , -0.0420887 , -0.02673123, ...,  0.01286111,
          0.03168869, -0.04308448]]], dtype=float32)>

In [27]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.02668208, -0.04364238, -0.0132379 , -0.04759381,  0.02433366,
         0.04223955,  0.02438809, -0.02111145,  0.02214866,  0.02147851,
        -0.01635029,  0.02316672,  0.00257295, -0.01045383,  0.00182381,
        -0.03824501, -0.02084833, -0.03200333,  0.04432194,  0.00345508,
        -0.03344315,  0.00302664, -0.03589224, -0.00785106, -0.03760349,
        -0.01461583, -0.04346351,  0.04947016,  0.01022666, -0.03233212,
        -0.03173289,  0.03702039,  0.00334319, -0.00160269,  0.04867227,
        -0.02920192, -0.04762019, -0.04855442, -0.03545962,  0.03768494,
         0.03061496, -0.03668695,  0.04799715, -0.01049116, -0.00307091,
         0.01951292,  0.03845507, -0.01260383, -0.01436658, -0.03285396,
         0.01564193,  0.02688355,  0.00654484, -0.03537101, -0.03820159,
         0.0235862 ,  0.04199525, -0.02069509,  0.01788125, -0.03597723,
        -0.01420254,  0.04441932,  0.04373297, -0.03789818,  0.01239746,
  

## Modelling a text dataset (running a bunch of experiments)

Now we've got a way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline), this is from Sklearn ML Map
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM Model (RNN)
* Model 3: GRU Model (RNN)
* Model 4: Bidirectional-LSTM (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pretrained feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6, but with 10% training data

### Model 0: getting a baseline

As with all machine learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experiments to build upon. 

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

> **NOTE:** It's common practice to use non-DL algorythms as a baseline model because of their speed and then later using DL to see if you can improve upon them.


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()), # convert words to numbers using TF-IDF
    ('clf', MultinomialNB()) # model the text
])

# Fit the pipeline to the data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [30]:
# Make predictions 
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

We could evaluate all of our model's predictions with different metrics every time. However this may bu cumbersom and will surely take some time.

Let's create one to compare our model's predcitions with truth labels using the following metrics:
* Accuracy
* Precision
* Recall
* F1-score

In [31]:
# Function to evaluate: accuracy, recall, precision, f1-score

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculate model accuracy, recall, precision and f1-score of a binary classification model.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100

  # Calculate precision, recall and f1-score using 'weighted' metrics
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

  model_results = {'accuracy': model_accuracy,
                   'precision': model_precision,
                   'recall': model_recall,
                   'f1': model_f1}

  return model_results

In [33]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: A simple dense model

In [34]:
# Create a tensorboard callback (need to create new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [53]:
# Build model with Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of numberized inputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to each vector
outputs = layers.Dense(1, activation='sigmoid')(x) # create the output layer, want binary output so sigmoid

model_1 = tf.keras.Model(inputs, outputs, name='model_1_dense')

In [54]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_2   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [55]:
# Compile model
model_1.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

In [56]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name='model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20230318-222551
Epoch 1/5
215/215 [==============================] - 8s 33ms/step - loss: 0.4641 - accuracy: 0.8848 - val_loss: 0.4831 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2709 - accuracy: 0.9266 - val_loss: 0.4597 - val_accuracy: 0.7927
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2035 - accuracy: 0.9415 - val_loss: 0.4759 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1675 - accuracy: 0.9502 - val_loss: 0.4996 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1444 - accuracy: 0.9561 - val_loss: 0.5279 - val_accuracy: 0.7782


In [57]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 5ms/step - loss: 0.5279 - accuracy: 0.7782


[0.5278857946395874, 0.778215229511261]

In [58]:
# Make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.31771472],
       [0.8005804 ],
       [0.99765426],
       [0.12841883],
       [0.03033534],
       [0.9631376 ],
       [0.85247153],
       [0.99908066],
       [0.9939902 ],
       [0.3964136 ]], dtype=float32)

In [59]:
# Single prediction
model_1_pred_probs[1]

array([0.8005804], dtype=float32)

In [60]:
# Convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [61]:
# Calculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 77.82152230971128,
 'precision': 0.7801652612118941,
 'recall': 0.7782152230971129,
 'f1': 0.7761410008648767}

In [62]:
import numpy as np
np.array(list(model_1_results.values()))> np.array(list(baseline_results.values()))

array([False, False, False, False])